# Analysis of DNA-MERFISH for CTP11

by Pu Zheng

2022.02.15

analysis for dataset:

\\10.245.74.158\Chromatin_NAS_0\20220215-P_brain_CTP11-1000_CTP12_from0208

This data is DNA of uncleared MERFISH RNA:
    \\10.245.74.158\Chromatin_NAS_0\20220208-P_brain_M1_nonclear


In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

16268


# 1. Pre-processing info

In [2]:
fov_param = {'data_folder':r'\\10.245.74.158\Chromatin_NAS_1\20220307-P_brain_CTP11_from_0303',
             'save_folder':r'\\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303',
             'experiment_type': 'DNA',
             'num_threads': 44,
             'correction_folder':r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210621-Corrections_lumencor_from_60_to_50',
             'shared_parameters':{
                     'single_im_size':[50,2048,2048],
                     'distance_zxy': [250, 108, 108],
                     'corr_channels':['750','647','561'],
                     'num_empty_frames': 0, 
                     'num_buffer_frames':0,
                     'corr_hot_pixel':True,
                     'corr_Z_shift':False,
                     'corr_bleed':True,
                     'min_num_seeds':10,
                     'max_num_seeds': 20000,
                     'spot_seeding_th': 1000,
                     'normalize_intensity_local':False,
                     'normalize_intensity_background':False,
                     'corr_gaussian_highpass':False,
                 }, 
             }

## 1.1 define required floders

In [3]:
save_folder = fov_param['save_folder']

save_filenames = [os.path.join(save_folder, _fl) for _fl in os.listdir(save_folder)
                  if _fl.split(os.extsep)[-1]=='hdf5']
# extract fov_id
save_fov_ids = [int(os.path.basename(_fl).split('.hdf5')[0].split('_')[-1]) for _fl in save_filenames]

debug = False

print(f"{len(save_filenames)} fovs detected")

segmentation_folder = os.path.join(save_folder, 'Segmentation')
if not os.path.exists(segmentation_folder):
    os.makedirs(segmentation_folder)
    print(f"Creating segmentation_folder: {segmentation_folder}")
else:
    print(f"Use segmentation_folder: {segmentation_folder}")

cand_spot_folder = os.path.join(save_folder, 'CandSpots')
if not os.path.exists(cand_spot_folder):
    os.makedirs(cand_spot_folder)
    print(f"Creating cand_spot_folder: {cand_spot_folder}")
else:
    print(f"Use cand_spot_folder: {cand_spot_folder}")

decoder_folder = cand_spot_folder.replace('CandSpots', 'Decoder')
if debug:
    _version = 0
    while os.path.exists(os.path.join(decoder_folder, f'v{_version}')):
        _version += 1
    decoder_folder = os.path.join(decoder_folder, f'v{_version}')
if not os.path.exists(decoder_folder):
    os.makedirs(decoder_folder)
    print(f"Creating decoder_folder: {decoder_folder}")
else:
    print(f"Use decoder_folder: {decoder_folder}")


158 fovs detected
Use segmentation_folder: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation
Use cand_spot_folder: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\CandSpots
Use decoder_folder: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder


In [4]:
pixel_sizes = np.array(fov_param['shared_parameters']['distance_zxy'])
single_im_size = np.array(fov_param['shared_parameters']['single_im_size'])
intensity_th = np.array(fov_param['shared_parameters']['spot_seeding_th'])

# 2. Partiton spots into cells

## 2.1 (For DNA-only) run segmentation

## (For DNA after dense MERFISH)

In [8]:
# MERFISH segmentation
merfish_segmentation_folder = r'\\mendel\Mendel_SSD3\MERFISH_Analysis\20220303-P_brain_M1_nonclear_adaptors\CellPoseSegment\features'
merfish_dapi_folder = r'\\10.245.74.158\Chromatin_NAS_0\20220303-P_brain_M1_nonclear_adaptors\Segmentation_Cellpose'
if not os.path.exists(merfish_dapi_folder):
    os.makedirs(merfish_dapi_folder)
# generate alignment
rna_data_folder = r'\\10.245.74.158\Chromatin_NAS_0\20220303-P_brain_M1_nonclear_adaptors'
rna_alignment_file = os.path.join(rna_data_folder, 'Alignment', '10x_positions_before.txt')
dna_alignment_file = os.path.join(fov_param['data_folder'], 'Alignment', '10x_positions_after_new.txt')
print(rna_alignment_file, '\n', dna_alignment_file)
print(os.path.exists(rna_alignment_file), os.path.exists(dna_alignment_file))
R, t = ia.correction_tools.alignment.align_manual_points(rna_alignment_file, dna_alignment_file,
                                                         save_folder=save_folder)

\\10.245.74.158\Chromatin_NAS_0\20220303-P_brain_M1_nonclear_adaptors\Alignment\10x_positions_before.txt 
 \\10.245.74.158\Chromatin_NAS_1\20220307-P_brain_CTP11_from_0303\Alignment\10x_positions_after_new.txt
True True
- Manually picked points aligned, rotation:
[[ 0.99999879  0.00155557]
 [-0.00155557  0.99999879]],
 translation:[-802.28134017  602.34215629]
-- rotation matrix saved to file:\\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\rotation
-- translation matrix saved to file:\\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\translation


## save DAPI image for RNA

In [9]:
from tqdm import tqdm
rna_fds, rna_fovs = ia.io_tools.data.get_folders(rna_data_folder)
ref_fd = rna_fds[0]

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 12
- Number of field of views: 158


In [10]:
overwrite_dapi = False
for _fov_id in tqdm(save_fov_ids):
    _dapi_savefile = os.path.join(merfish_dapi_folder, rna_fovs[_fov_id].replace('.dax', '_Dapi.npy'))
    if overwrite_dapi or not os.path.exists(_dapi_savefile):
        # load 
        _im = ia.visual_tools.DaxReader(os.path.join(ref_fd, rna_fovs[_fov_id])).loadAll()
        _dapi_im = _im[4::5]
        # save
        np.save(_dapi_savefile.split('.npy')[0], _dapi_im)

100%|████████████████████████████████████████████████████████████████████████████| 158/158 [00:00<00:00, 934.21it/s]


In [11]:
import multiprocessing as mp
# savefile for segmentations
_total_seg_save_file = os.path.join(segmentation_folder, 'full_segmentation.hdf5')
# required parameters
microscope_file = r'\\mendel\pu_documents\Merfish_analysis\Merfish_Analysis_Scripts\merlin_parameters\microscope\storm6_microscope.json'
Zcoords = np.arange(0,12.5,0.25) # z-coordinates of all z-planes in this experiment
seg_align_params = {}
overwrite_segmentation = False
plot_segmentation = True

In [12]:
# initiate locks
_manager = mp.Manager()
# savefile lock
_segmentation_savefile_lock = _manager.RLock()
_seg_align_args = []
# prepare kwargs
for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):
    # segmentation filename
    _segmentation_filename = os.path.join(segmentation_folder, 
        os.path.basename(_save_filename).replace('.hdf5', '_Segmentation.npy') )
    _rna_feature_filename = os.path.join(merfish_segmentation_folder, f"feature_data_{_fov_id}.hdf5")
    _rna_dapi_filename = os.path.join(merfish_dapi_folder, 
                                      os.path.basename(_save_filename).replace('.hdf5', '_Dapi.npy'))

    _args = (_fov_id, Zcoords, _rna_feature_filename, _rna_dapi_filename,
             _save_filename, microscope_file, R, 
             _total_seg_save_file, True, _segmentation_savefile_lock, 
             seg_align_params, plot_segmentation, overwrite_segmentation, False, False, True,
             )
    _seg_align_args.append(_args)
print(len(_seg_align_args))

158


In [12]:
%%time
from ImageAnalysis3.segmentation_tools.cell import _batch_align_segmentation
# Multiprocessing
print(f"- Start multiprocessing segmentation alignment", end=' ')
_start_time = time.time()
with mp.Pool(12) as _seg_pool:
    # start multiprocessing
    _seg_pool.starmap(_batch_align_segmentation, _seg_align_args, chunksize=1)
    # close multiprocessing
    _seg_pool.close()
    _seg_pool.join()
    _seg_pool.terminate()
print(f"finish in {time.time()-_start_time:.3f}s. ")

2022-03-18 11:18:15,614 [INFO] WRITING LOG OUTPUT TO C:\Users\puzheng\.cellpose\run.log
- Start multiprocessing segmentation alignment finish in 2584.533s. 
Wall time: 43min 35s


## 2.2 Partition DNA-MERFISH spots

In [13]:
from ImageAnalysis3.classes.partition_spots import Spots_Partition
from ImageAnalysis3.classes.preprocess import Spots3D
from ImageAnalysis3.figure_tools import plot_partition
import pandas as pd

In [14]:
%%time
from ImageAnalysis3.segmentation_tools.cell import Align_Segmentation
from ImageAnalysis3.io_tools.spots import FovCell2Spots_2_DataFrame
reload(segmentation_tools.cell)
reload(io_tools.spots)
search_radius = 3
overwrite_cand_spots = False

_partition_args = []

for _fov_id, _save_filename in zip(save_fov_ids, save_filenames):

    
    # savename
    _cand_spot_filename = os.path.join(cand_spot_folder,
                                      os.path.basename(_save_filename).replace('.hdf5', f'_CandSpots.csv') )
    
    # load segmentation label matrix and uids
    _align_seg = Align_Segmentation('', '', _save_filename, '', np.array([]))
    _align_seg._load(_total_seg_save_file)
    seg_label, fovcell_2_uid = _align_seg.dna_mask, _align_seg.fovcell_2_uid
    
    # load cand_spots_list
    with h5py.File(_save_filename, "r", libver='latest') as _f:
        _grp = _f['combo']
        combo_spots_list = [_spots[_spots[:,0]>0] for _spots in _grp['spots'][:]]
        combo_bits = _grp['ids'][:]
        combo_channels = [_ch.decode() for _ch in _grp['channels'][:]]
    # partition args
    _args = (
        _fov_id, seg_label, fovcell_2_uid, combo_spots_list, combo_bits, combo_channels,
        _cand_spot_filename, search_radius, pixel_sizes,
        True, False, False, True,
    )
    _partition_args.append(_args)
    
print(len(_partition_args))

-- loading segmentation info from fov:0 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:1 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:2 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:3 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:4 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:5 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:6 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_fro

-- loading segmentation info from fov:55 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:56 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:57 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:58 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:59 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:60 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:61 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CT

-- loading segmentation info from fov:110 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:111 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:112 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:113 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:114 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:115 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Segmentation\full_segmentation.hdf5
-- loading segmentation info from fov:116 into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_b

In [15]:
%%time
from ImageAnalysis3.classes.partition_spots import _batch_partition_spots
# Multiprocessing
print(f"- Start multiprocessing spot partitioning", end=' ')
_start_time = time.time()
with mp.Pool(12) as _partition_pool:
    # start multiprocessing
    _partition_pool.starmap(_batch_partition_spots, _partition_args, chunksize=1)
    # close multiprocessing
    _partition_pool.close()
    _partition_pool.join()
    _partition_pool.terminate()
print(f"finish in {time.time()-_start_time:.3f}s. ")

- Start multiprocessing spot partitioning finish in 895.427s. 
Wall time: 14min 55s


# 3. Decoding of DNA-MERFISH

## 3.1 load codebook

In [5]:
import pandas as pd
codebook_filename = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-mouse-genome-1000_codebook.csv'
#
codebook_df = pd.read_csv(codebook_filename, header=0)
codebook_df

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
984,X:163740000-163760000,1058,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,X,53
985,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
986,X:168740000-168760000,1060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,55


## 3.2 load spot files

In [6]:
with h5py.File(save_filenames[0], "r", libver='latest') as _f:
    _grp = _f['combo']
    combo_channels = [_ch.decode() for _ch in _grp['channels'][:]]
    combo_ids = _grp['ids'][:]
    
bit_2_channel = {_b:_ch for _b,_ch in zip(combo_ids, combo_channels)}

## 3.3 test decode one cell

In [33]:
%%time
from tqdm import tqdm
from ImageAnalysis3.classes import decode
reload(decode)

overwrite_decoder = False
return_decoder = True
load_exist = True

pair_search_radius = 300

decode_args = []
for _fov_id, _save_filename in tqdm(zip(save_fov_ids, save_filenames)):
    
    #print(f"Prepare decoding args for fov: {_fov_id}")
    if _fov_id != 20:
        continue
    
    # load fov_df
    cand_spot_filename = os.path.join(cand_spot_folder,
                                      os.path.basename(_save_filename).replace('.hdf5', f'_CandSpots.csv') )
    
    if os.path.isfile(cand_spot_filename):
        _fov_spots_df = pd.read_csv(cand_spot_filename)
    else:
        continue
        
    for _cell_id in np.unique(_fov_spots_df['cell_id']):
        # get decoder filename
        _decoder_filename = os.path.join(decoder_folder, f'Fov-{_fov_id}_Cell-{_cell_id}_Decoder.hdf5')
        
        #if os.path.exists(_decoder_filename):
        #    continue
        # get cell_df
        _cell_spots_df =_fov_spots_df[_fov_spots_df['cell_id']==_cell_id]

        _args = (_cell_spots_df, codebook_df, _decoder_filename, 
                 False, True, bit_2_channel,
                 pixel_sizes, 2, 0.1, 
                 pair_search_radius, -1, 1, 5, 0, -25, 
                 True, overwrite_decoder, return_decoder, False) 
        # append
        decode_args.append(_args)

print(len(decode_args))

158it [00:02, 67.38it/s]

161
Wall time: 2.35 s


In [42]:
# test run one cell
%matplotlib inline
reload(decode)
_cell_ind = 11
decoder = decode.batch_decode_DNA(*decode_args[_cell_ind])

- Loading decoder info from file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
Creating DNA_Merfish_Decoder class.
- Load chr_2_assigned_tuple_list from file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
-- 20 chromosome loaded.
- Load chr_2_zxys_list from file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
-- 20 chromosome loaded.
- Load chr_2_chr_centers from file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
-- 20 chromosome loaded.
- spot_groups and spot_usage already exist, skip.
- split found tuples into chromosomes. 
- generate randomized spot pairs 
- generate chr_2_kdtree. 
- Saving decoder into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
-- save cand_spots into: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CT

E:\Users\puzheng\AppData\anaconda3\lib\site-packages\pandas\core\generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['uid', 'homolog', 'channel_0', 'channel_1', 'channel_2', 'start', 'end',
       'chr'],
      dtype='object')]

  encoding=encoding,


-- save spot_groups into: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
- Directly return assigned homolog results.
- Save chr_2_assigned_tuple_list into file: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Fov-20_Cell-13_Decoder.hdf5
-- skip :chr_2_assigned_tuple_list/1/0
-- skip :chr_2_assigned_tuple_list/1/1
-- skip :chr_2_assigned_tuple_list/10/0
-- skip :chr_2_assigned_tuple_list/10/1
-- skip :chr_2_assigned_tuple_list/11/0
-- skip :chr_2_assigned_tuple_list/11/1
-- skip :chr_2_assigned_tuple_list/12/0
-- skip :chr_2_assigned_tuple_list/12/1
-- skip :chr_2_assigned_tuple_list/13/0
-- skip :chr_2_assigned_tuple_list/13/1
-- skip :chr_2_assigned_tuple_list/14/0
-- skip :chr_2_assigned_tuple_list/14/1
-- skip :chr_2_assigned_tuple_list/15/0
-- skip :chr_2_assigned_tuple_list/15/1
-- skip :chr_2_assigned_tuple_list/16/0
-- skip :chr_2_assigned_tuple_list/16/1
-- skip :chr_2_assigned_tuple_list/17/0
-- skip :chr

In [72]:
region_ids = []
region_coords = []
for _g in decoder.spot_groups:
    region_ids.append(_g.tuple_id)
    region_coords.append(_g.centroid_spot().to_positions()[0]/1000)
region_ids = np.array(region_ids)
region_coords = np.array(region_coords)

In [49]:
save_figure = True
#figure_folder = os.path.join(save_folder, 'Figures_final')
figure_folder = os.path.join(decoder_folder, 'Figures_final')
if not os.path.exists(figure_folder):
    print(f"Create figure_folder: {figure_folder}")
    os.makedirs(figure_folder)
else:
    print(f"Use figure_folder: {figure_folder}")

Use figure_folder: \\mendel\Mendel_SSD4\Pu_Temp\20220307-P_brain_CTP11_from_0303\Decoder\Figures_final


In [66]:
%matplotlib notebook
plt.style.use('dark_background')

def rotate(angle):
     ax.view_init(azim=angle)
        
from matplotlib import animation

from matplotlib.cm import Reds, Blues, Spectral
fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection='3d')
ax.set_facecolor([0,0,0,0])

ax.scatter(region_coords[:,1], region_coords[:,2], region_coords[:,0],
           cmap=Spectral, 
           c=region_ids,
           alpha=0.95,
           s=1, )
ax.grid(False)
ax.xaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.yaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.zaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))

angle = 3
ani = animation.FuncAnimation(fig, rotate, frames=np.arange(0, 360, angle), interval=50)
ani.save(os.path.join(figure_folder, 
                      os.path.basename(decoder.savefile).replace('.hdf5', '_DecodeGroups.gif')), 
         writer=animation.PillowWriter(fps=20))


<IPython.core.display.Javascript object>

In [84]:
sel_inds = np.where(np.array(region_ids)=)[0]
sel_inds

array([794, 806], dtype=int64)

In [68]:
chr_2

(array([1062, 2316], dtype=int64),)

In [91]:
%matplotlib notebook
plt.style.use('dark_background')
sel_reg = 266
sel_inds = np.where((np.array(region_ids)<=sel_reg) & (np.array(region_ids)>=208))[0]
sel_inds

def rotate(angle):
     ax.view_init(azim=angle)
        
from matplotlib import animation

from matplotlib.cm import Reds, Blues, Spectral
fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection='3d')
ax.set_facecolor([0,0,0,0])

ax.scatter(region_coords[sel_inds,1], region_coords[sel_inds,2], region_coords[sel_inds,0],
           cmap='bwr', 
           c=region_ids[sel_inds],
           alpha=0.95,
           s=1, )
ax.grid(False)
ax.xaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.yaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.zaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))

angle = 3
ani = animation.FuncAnimation(fig, rotate, frames=np.arange(0, 360, angle), interval=50)
ani.save(os.path.join(figure_folder, 
                      os.path.basename(decoder.savefile).replace('.hdf5', f'_DecodeGroups_reg-{sel_reg}.gif')), 
         writer=animation.PillowWriter(fps=20))


<IPython.core.display.Javascript object>

In [93]:
%matplotlib notebook
def rotate(angle):
     ax.view_init(azim=angle)
        
from matplotlib import animation

from matplotlib.cm import Reds, Blues, Spectral
fig = plt.figure(dpi=150)
ax = fig.add_subplot(projection='3d')

_chr_name = '4'
_zxys_list = decoder.chr_2_zxys_list[_chr_name]/1000

for _ichr, _zxys in enumerate(_zxys_list):
    
    ax.scatter(_zxys[:,1], _zxys[:,2], _zxys[:,0],
               cmap=Spectral, 
               c=Spectral(_ichr/(len(_zxys_list)+1)),
               alpha=0.7,
               s=3)

    ax.plot(_zxys[:,1], _zxys[:,2], _zxys[:,0], linewidth=0.5,
            alpha=0.7,
            color = Spectral( _ichr/(len(_zxys_list)+1) ) )
    
ax.grid(False)
ax.xaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.yaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.zaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))

angle = 3
ani = animation.FuncAnimation(fig, rotate, frames=np.arange(0, 360, angle), interval=50)
ani.save(os.path.join(figure_folder, 
                      os.path.basename(decoder.savefile).replace('.hdf5', f'_Picked_chr-{_chr_name}.gif')), 
         writer=animation.PillowWriter(fps=20))


plt.show()

<IPython.core.display.Javascript object>

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.


### decoder

## visualize decoded spots

In [172]:
if not os.path.exists(decoder_folder):
    os.makedirs(decoder_folder)
    print(f"Creating decoder_folder: {decoder_folder}")
else:
    print(f"Use decoder_folder: {decoder_folder}")

decode_figure_folder = os.path.join(decoder_folder, 'Figures')
if not os.path.exists(decode_figure_folder):
    os.makedirs(decode_figure_folder)
    print(f"Creating decode_figure_folder: {decode_figure_folder}")
else:
    print(f"Use decode_figure_folder: {decode_figure_folder}")

Use decoder_folder: \\mendel\Mendel_SSD1\Pu_Temp\20220215-P_brain_CTP11-1000_CTP12_from0208\Decoder\Tests
Use decode_figure_folder: \\mendel\Mendel_SSD1\Pu_Temp\20220215-P_brain_CTP11-1000_CTP12_from0208\Decoder\Tests\Figures


In [ ]:
%matplotlib notebook

def rotate(angle):
     ax.view_init(azim=angle)
        
from matplotlib import animation

from matplotlib.cm import Reds, Blues, Spectral
fig = plt.figure(dpi=100)
ax = fig.add_subplot(projection='3d')

_zxys_list = decoder.chr_2_zxys_list['2']

for _ichr, _zxys in enumerate(_zxys_list):
    
    ax.scatter(_zxys[:,1], _zxys[:,2], _zxys[:,0],
               #cmap=Spectral, 
               color=Spectral(_ichr/(len(_zxys_list)+1)),
               #c=homolog_labels,
               alpha=0.7,
               s=3)

    ax.plot(_zxys[:,1], _zxys[:,2], _zxys[:,0], linewidth=0.5,
            alpha=0.7,
            color = Spectral( _ichr/(len(_zxys_list)+1) ) )
    
fig.show()

## 3.4 process all

In [8]:
%%time
# old version
import multiprocessing as mp

print(len(decode_args))
with mp.Pool(44) as decode_pool:
    decode_results = decode_pool.starmap(decode.batch_decode_DNA, decode_args, chunksize=1)
    decode_pool.close()
    decode_pool.join()
    decode_pool.terminate()

12750
Wall time: 5h 14min 16s


# 5. Summarize decoder

Please goto the next jupyter:

Chromatin_Analysis_Scripts/Tissue_DNA-FISH/CTP12_marker-gene/20220307-PostAnalysis_CellType.ipynb